In [6]:
#packages
#to install in our environment : conda install -c conda-forge ipywidgets jupyter nbextension enable --py widgetsnbextension
import pandas as pd
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

/var/folders/ll/9ks4sslj4hj9kg36pxlnc3xc0000gn/T/ipykernel_56249/446153002.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

In [7]:
#function to check if strin is contained and update dico
def isinside1(test_string,test_list):
    global dico
    res = [ele for ele in test_list if(ele in test_string)]
    #print(res)
    if res:
        for ele in res:
            i= test_list.index(ele)
            dico["occurences"].loc[i]= dico["occurences"].loc[i]+1
        return 1
    return 0

def isinside2(test_string,test_list):
    if any(ext in test_string for ext in test_list):
        return 1
    return 0

In [8]:
#get dico
dico= pd.read_pickle("./Reduced_Final_Dictionary.pkl")
dico["occurences"]=0
dico.head()

,word,occurences
0,Janky,0
1,slumpin',0
2,yayeeyay,0
3,ho-bag,0
4,hork,0


In [10]:
#check strings lenths and describe dico
dico["strlen"]= dico["word"].apply(lambda x : len(x))
dico.describe()

,occurences,strlen
count,16339.0,16339.000000
mean,0.0,9.643858
std,0.0,5.264567
min,0.0,1.000000
25%,0.0,6.000000
50%,0.0,9.000000
75%,0.0,12.000000
max,0.0,143.000000


In [11]:
#make list out of dico
dicolist= dico["word"].unique().tolist()
#dicolist

In [12]:
#importing dataset sample

#Jules : 
#path=r'/Users/jules/ADA_project_data/Sample_cleaned_1Mio.json.bz2'

#Nico: '/Users/nicolasantacroce/Desktop/Desktop/EPFL/EPFL MA1/Applied Data Analysis/Sample.json.bz2'
df= pd.read_json('/Users/nicolasantacroce/Desktop/Desktop/EPFL/EPFL MA1/Applied Data Analysis/Sample.json.bz2',compression="bz2",lines=True)

In [13]:
df.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2015-11-11-109291,They'll call me lots of different things. Libe...,Chris Christie,[Q63879],2015-11-11 00:55:12,1,"[[Chris Christie, 0.7395], [Bobby Jindal, 0.15...",[http://thehill.com/blogs/ballot-box/259760-ch...,E
1,2015-11-04-105046,"The choices are not that easy,",Dr. John,"[Q511074, Q54593093]",2015-11-04 18:13:06,2,"[[Dr. John, 0.5531], [None, 0.4469]]",[http://delawareonline.com/story/news/health/2...,E
2,2015-09-11-070666,It's kind of the same way it's been with the R...,Niklas Kronwall,[Q722939],2015-09-11 19:54:00,1,"[[Niklas Kronwall, 0.7119], [None, 0.2067], [H...",[http://redwings.nhl.com/club/news.htm?id=7787...,E
3,2015-01-12-082489,"We're now going back to the frozen tundra, and...",Frances McDormand,[Q204299],2015-01-12 01:40:00,3,"[[Frances McDormand, 0.484], [None, 0.4495], [...",[http://feeds.people.com/~r/people/headlines/~...,E
4,2015-11-09-033345,I had a chuckle: They were showing a video of ...,Kris Draper,[Q948695],2015-11-09 00:57:45,3,"[[Kris Draper, 0.8782], [None, 0.1043], [Serge...",[http://ca.rd.yahoo.com/sports/rss/nfl/SIG=13u...,E


In [14]:
#separating quotes (really long so we try it on the first 10000)
df2= df.loc[0:9999]
df2["colloquial"]= df2["quotation"].progress_apply(lambda x : isinside1(x,dicolist))

  0%|          | 0/10000 [00:00<?, ?it/s]

/Users/nicolasantacroce/opt/anaconda3/envs/adaenv/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/var/folders/ll/9ks4sslj4hj9kg36pxlnc3xc0000gn/T/ipykernel_56249/2406884457.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["colloquial"]= df2["quotation"].progress_apply(lambda x : isinside1(x,dicolist))


In [15]:
df2.describe()

,numOccurrences,colloquial
count,10000.000000,10000.000000
mean,3.260400,0.955600
std,18.155279,0.205992
min,1.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,2.000000,1.000000
max,1125.000000,1.000000


In [20]:
#removing words present in more than 0.1% of quotes
tresh= 0.001
dico["occurence_fraction"]= dico["occurences"]/df2["colloquial"].count()
dico.describe()

,occurences,strlen,occurence_fraction
count,16339.000000,16339.000000,16339.000000
mean,3.178652,9.643858,0.000318
std,74.463508,5.264567,0.007446
min,0.000000,1.000000,0.000000
25%,0.000000,6.000000,0.000000
50%,0.000000,9.000000,0.000000
75%,0.000000,12.000000,0.000000
max,6573.000000,143.000000,0.657300


In [21]:
#creating new dictionary without most occuring words
dico2= dico[dico["occurence_fraction"]<tresh]
dicolist2= dico2["word"].unique().tolist()
dico2.describe()

,occurences,strlen,occurence_fraction
count,16142.000000,16142.000000,16142.000000
mean,0.074960,9.715834,0.000007
std,0.545739,5.252921,0.000055
min,0.000000,1.000000,0.000000
25%,0.000000,6.000000,0.000000
50%,0.000000,9.000000,0.000000
75%,0.000000,12.000000,0.000000
max,9.000000,143.000000,0.000900


In [22]:
#re-evaluating quotes with reduced dictionary
df2["colloquial"]= df2["quotation"].progress_apply(lambda x : isinside2(x,dicolist2))

  0%|          | 0/10000 [00:00<?, ?it/s]

/var/folders/ll/9ks4sslj4hj9kg36pxlnc3xc0000gn/T/ipykernel_56249/745375064.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["colloquial"]= df2["quotation"].progress_apply(lambda x : isinside2(x,dicolist2))


In [23]:
df2.describe()

,numOccurrences,colloquial
count,10000.000000,10000.000000
mean,3.260400,0.105200
std,18.155279,0.306826
min,1.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,0.000000
max,1125.000000,1.000000


In [56]:
df_colloquials= df2[df2["colloquial"]==1]
df_colloquials[["quotation"]].sample()["quotation"].tolist()

['It seems to be that every time effective investigators like the ones we work with begin to turn over rocks, they find creepy crawly things,']

In [57]:
dico2.sample(20)

,word,occurences,strlen,occurence_fraction
11994,Sweet Ass Maria Sally Muzzone,0,29,0.0
5287,Dutch Oven,0,10,0.0
359,coucher,0,7,0.0
15428,scrub up well,0,13,0.0
8893,Chiparati,0,9,0.0
4861,speen,0,5,0.0
14413,kick off,0,9,0.0
6165,D-ware,0,6,0.0
11396,ashey,0,5,0.0
10276,fo' shizzle,0,11,0.0


In [63]:
dico2[dico2["word"]=="Fuck"]

,word,occurences,strlen,occurence_fraction
